In [2]:
import os
import sys
import argparse
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc
import traceback
from statistics import median
from statistics import mean
from statistics import mode
from statistics import stdev
from scipy import stats
from scipy import signal
import portion as P
import math
import random
import itertools

database = "/home/wmnlab/D/database/"
dates = [
         "2023-02-04",
         "2023-02-04#1",
         "2023-02-04#2",
         ]
devices = [
    "qc01",
    "qc02",
    "qc03",
]
schemes = [
    "B3",
    "B7",
    "B8",
    # "All@qc01",
    # "All@qc02",
    # "All@qc03",
]
exps = {  # experiment_name: (number_of_experiment_rounds, list_of_experiment_round)
            # If the list is None, it will not list as directories.
            # If the list is empty, it will list all directories in the current directory by default.
            # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    "_Bandlock_Udp_B3_B7_B8_RM500Q": (2, ["#01", "#02"]),
    "_Bandlock_Udp_all_RM500Q": (2, ["#01", "#02"]),
    # "_Bandlock_Udp_B3_B7_B8_RM500Q": (1, ["#01",]),
    # "_Bandlock_Udp_all_RM500Q": (1, ["#01",]),
}

def makedir(dirpath, mode=0):  # mode=1: show message, mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)

# Testing

In [4]:
devices = ['qc01','qc02','qc03']
schemes = ['B3','B7','B8']
dfs = []
testpath = [
    '/home/wmnlab/D/database/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/udp_dnlk_loss_latency_ho.pkl',
    '/home/wmnlab/D/database/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02/#01/data/udp_dnlk_loss_latency_ho.pkl',
    '/home/wmnlab/D/database/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03/#01/data/udp_dnlk_loss_latency_ho.pkl',
]
for i, (dev, schm) in enumerate(zip(devices, schemes)):
    dfs.append(pd.read_pickle(testpath[i]))
for i, (dev, schm) in enumerate(zip(devices, schemes)):
    display(dfs[i])

,sequence_num,Timestamp,lost,excl,latency,transmit_time,arrival_time,ho_index,ho_state,ho_type,ho_type1,ho_type2,ho_fcause,ho_interrupt,ho_ePCI,ho_earfcn,ho_nrPCI,ho_ePCI1,ho_earfcn1,ho_nrPCI1
0,10001,2023-02-04 16:16:36.846746,False,False,0.008288,2023-02-04 16:16:36.846760,2023-02-04 16:16:36.855034,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,10002,2023-02-04 16:16:36.848746,False,False,0.011733,2023-02-04 16:16:36.848758,2023-02-04 16:16:36.860479,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,10003,2023-02-04 16:16:36.850746,False,False,0.009779,2023-02-04 16:16:36.850755,2023-02-04 16:16:36.860525,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,10004,2023-02-04 16:16:36.852746,False,False,0.007780,2023-02-04 16:16:36.852756,2023-02-04 16:16:36.860526,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,10005,2023-02-04 16:16:36.854747,False,False,0.005780,2023-02-04 16:16:36.854756,2023-02-04 16:16:36.860527,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264515,274516,2023-02-04 16:25:25.901468,False,False,0.048541,2023-02-04 16:25:25.901475,2023-02-04 16:25:25.950009,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264516,274517,2023-02-04 16:25:25.903468,False,False,0.052752,2023-02-04 16:25:25.903475,2023-02-04 16:25:25.956220,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264517,274518,2023-02-04 16:25:25.905468,False,False,0.056539,2023-02-04 16:25:25.905475,2023-02-04 16:25:25.962007,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264518,274519,2023-02-04 16:25:25.907468,False,False,0.054542,2023-02-04 16:25:25.907475,2023-02-04 16:25:25.962010,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


,sequence_num,Timestamp,lost,excl,latency,transmit_time,arrival_time,ho_index,ho_state,ho_type,ho_type1,ho_type2,ho_fcause,ho_interrupt,ho_ePCI,ho_earfcn,ho_nrPCI,ho_ePCI1,ho_earfcn1,ho_nrPCI1
0,10001,2023-02-04 16:16:36.846746,False,False,0.005360,2023-02-04 16:16:36.846765,2023-02-04 16:16:36.852106,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,10002,2023-02-04 16:16:36.848746,False,False,0.003361,2023-02-04 16:16:36.848761,2023-02-04 16:16:36.852107,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,10003,2023-02-04 16:16:36.850746,False,False,0.006130,2023-02-04 16:16:36.850758,2023-02-04 16:16:36.856876,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,10004,2023-02-04 16:16:36.852746,False,False,0.009315,2023-02-04 16:16:36.852759,2023-02-04 16:16:36.862061,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,10005,2023-02-04 16:16:36.854747,False,False,0.007356,2023-02-04 16:16:36.854760,2023-02-04 16:16:36.862103,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264515,274516,2023-02-04 16:25:25.901468,False,False,0.051039,2023-02-04 16:25:25.901478,2023-02-04 16:25:25.952507,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264516,274517,2023-02-04 16:25:25.903468,False,False,0.055100,2023-02-04 16:25:25.903479,2023-02-04 16:25:25.958568,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264517,274518,2023-02-04 16:25:25.905468,False,False,0.053103,2023-02-04 16:25:25.905478,2023-02-04 16:25:25.958571,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264518,274519,2023-02-04 16:25:25.907468,False,False,0.057169,2023-02-04 16:25:25.907479,2023-02-04 16:25:25.964637,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


,sequence_num,Timestamp,lost,excl,latency,transmit_time,arrival_time,ho_index,ho_state,ho_type,ho_type1,ho_type2,ho_fcause,ho_interrupt,ho_ePCI,ho_earfcn,ho_nrPCI,ho_ePCI1,ho_earfcn1,ho_nrPCI1
0,10001,2023-02-04 16:16:36.846746,False,False,0.006068,2023-02-04 16:16:36.846769,2023-02-04 16:16:36.852814,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,10002,2023-02-04 16:16:36.848746,False,False,0.009598,2023-02-04 16:16:36.848765,2023-02-04 16:16:36.858344,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,10003,2023-02-04 16:16:36.850746,False,False,0.013595,2023-02-04 16:16:36.850762,2023-02-04 16:16:36.864341,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,10004,2023-02-04 16:16:36.852746,False,False,0.011639,2023-02-04 16:16:36.852763,2023-02-04 16:16:36.864385,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,10005,2023-02-04 16:16:36.854747,False,False,0.009640,2023-02-04 16:16:36.854763,2023-02-04 16:16:36.864387,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264592,274593,2023-02-04 16:25:26.055475,False,False,0.050201,2023-02-04 16:25:26.055489,2023-02-04 16:25:26.105676,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264593,274594,2023-02-04 16:25:26.057475,False,False,0.054151,2023-02-04 16:25:26.057489,2023-02-04 16:25:26.111626,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264594,274595,2023-02-04 16:25:26.059475,False,False,0.052158,2023-02-04 16:25:26.059489,2023-02-04 16:25:26.111633,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
264595,274596,2023-02-04 16:25:26.061475,False,False,0.050160,2023-02-04 16:25:26.061490,2023-02-04 16:25:26.111635,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
